## 定义计算一个点是否在一个多边形内部的函数

In [6]:
def point_in_polygon(point, polygon):
    """
    判断一个点是否在多边形内部
    
    参数:
    point (tuple): 待测点的坐标 (x, y)
    polygon (list): 多边形顶点坐标列表 [(x1, y1), (x2, y2), ...]
    
    返回:
    bool: True 表示点在多边形内部, False 表示点在多边形外部
    """
    x, y = point
    inside = False
    
    # 遍历多边形的每条边
    for i in range(len(polygon)):
        p1 = polygon[i]
        p2 = polygon[(i + 1) % len(polygon)]
        
        # 检查射线是否与边相交
        if (p1[1] > y) != (p2[1] > y):
            at = (p2[0] - p1[0]) * (y - p1[1]) / (p2[1] - p1[1]) + p1[0]
            if at > x:
                inside = not inside
    
    return inside

# 多边形顶点坐标
polygon = [(0, 0), (0, 10), (5, 5), (10, 10), (11, 10), (11, 1)]

# 测试点坐标
point1 = (5, 6)  # 在多边形内部
point2 = (5, 4) # 在多边形外部
point3 = (8, 0.73)
point4 = (8, 0.71)

print(point_in_polygon(point1, polygon))  # True
print(point_in_polygon(point2, polygon))  # False
print(point_in_polygon(point3, polygon))  # False
print(point_in_polygon(point4, polygon))  # True

False
True
True
False


## Read Transaction Record

In [34]:
import csv
import sys
import os
from shapely.geometry import Polygon, Point

current_path = os.getcwd()
print(current_path)
csv.field_size_limit(sys.maxsize)
# 打开CSV文件

new_csv_path = '../../data/house_belonging.csv'

with open('../../data/BeijingHousingPrices_preprocessed.csv', newline='') as csvfile, \
     open(new_csv_path, mode='w', newline='') as new_csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    csvwriter = csv.writer(new_csvfile, delimiter=',')
    headers = next(csvreader)
    new_headers = headers + ['belonging']
    csvwriter.writerow(new_headers)
    lng_index = headers.index('Lng')
    lat_index = headers.index('Lat')
    i = 0
    for row in csvreader:
        lng = row[lng_index]
        lat = row[lat_index]
        point = Point(float(lng), float(lat))
        i = i + 1
        inside = False
        idx = 0
        with open('../../data/Beijing_geo.csv', newline='') as record:
            recordreader = csv.reader(record, delimiter=',')
            record_headers = next(recordreader)
            geometry_index = record_headers.index('geometry')
            new_id_index = record_headers.index('new_id')
            for record_row in recordreader:
                geometry = record_row[geometry_index]
                new_id = record_row[new_id_index]
                if "MULTIPOLYGON" in geometry:
                    idx = new_id
                elif "POLYGON" in geometry:
                    # 处理POLYGON
                    polygon = []
                    geometry_str_trimmed = geometry.replace("POLYGON ((", "").replace("))", "")
                    points_str = geometry_str_trimmed.split(", ")
                    for point_str in points_str:
                        x_str, y_str = point_str.split(" ")
                        polygon.append((float(x_str), float(y_str)))
                    poly_obj = Polygon(polygon)
                    if poly_obj.contains(point):
                        inside = True
                        idx = new_id
                        break
            new_row = row + [idx]
            csvwriter.writerow(new_row)
            if inside == False:
                print("not found")
    

/Users/serendipity/Desktop/教材/NUS暑研/SWS3019-2024/code/Question 1


KeyboardInterrupt: 